# (Denz and Timmesfeld, 2023) Visualizing the (Causal) Effect of a Continuous Variable on a Time-To-Event Outcome

[Link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10392888/)

# Abstract

In studies with time-to-event outcomes, the most popular visualization approach is depicting survival curves stratified by the variable of interest. This approach cannot be used when the variable of interest is continuous.

Simple workarounds, such as categorizing the continuous covariate and plotting survival curves for each category, can result in misleading depictions of the main effects. Instead, we propose a new graphic, the survival area plot, to directly depict the survival probability over time and as a function of a continuous covariate simultaneously

Through the use of g-computation, those estimates can be adjusted for confounding without additional effort, allowing a causal interpretation under the standard causal identifiability assumptions.

The `contsurvplot` R-package contains all methods discussed in this paper.

# Introduction

Visualization is a crucial aspect when reporting the main results of any study. When the aim of the study is to estimate a causal effect of one variable on a specific endpoint, a clear graphical display of the effect can make the results more accessible for the reader.

With a time-to-event endpoint, such as th time until recurrence of a disease or death, the most popular approach is to draw Kaplan-Meier survival curves, or confuonder-adjusted alternatives, for each group of interest. This, however, only works if there is a discrete number of groups. In reality, the variable of interest may be continuous, making the use of Kaplan-Meier survival curves impossible.

We propose a new type of graphic, the survival area plot. The survival area plot directly extends the standard Kaplan-Meier plot to the continuous case by depicting the survival probability as a function of time and a continuous covariate through the use of a color-scaled area. We use g-computation to obtain the required probability estimates, allowing the plot to be adjusted for confuonders. Under the standard causal identifiability assumptions, the resulting survival probabilities can be interpreted as counterfactual quantities, resulting in a display of the causal effect of the continuous variable on the time-to-event outcome.




# Methods

## Target Estimand

Need to first define what "the causal effect of a continuous variable on a time-to-event outcome" is.

FIrst, we discuss the target estimand in the simpler case of a dichotomous variable.

Let $Z$ denote the variable of interest with $Z\in\{0,1\}$. Let $T$ be the time until the occurence of an event. In ths setup, each individual has two potential failure times: $T^{Z=0}$ and $T^{Z=1}$.

Our goal is to estimate the population-level counterfactual survival function under both $Z=0$ and $Z=1$, defined as:

$$S_z(t)=E(I(T^{Z=z}>t))$$

counterfactual survival probability: the survival probability that would have been observed at the time $t$ if the value of $Z$ of all individuals in the population of interest had been set to $z$.

The difference between treatment-specific counterfactual quantities or their ratio may be used to define average causal effects.

It is straightforward to generalize this notation to the continuous case. Let $Z$ be a continuous variable that can take any real values $Z\subseteq\mathbb{R}$. Instead of two potential failure times, there is now an uncountably infinite set of potential failure times $T^{Z=z}$ for each individual.

## Estimation using $g$-computation

Suppose that setting $Z$ to $z$ corresponds to a well-defined intervention. If $Z$ was randomly assigned, estimating $S_z(t)$ is "only" an issue of interpolation. A suitable model is necessary to estimate this function. A popular alternative (to Kaplan-Meier estimator, which does not work for continuous treatments) is the Cox proportional hazards regression model (1972), given by:

$$h(t)=h_0(t)\exp(\beta Z)$$

where $\beta$ is the coefficient of the continuous variable and $h_0(t)$ is the baseline-hazard function, which can be estimated from the data. This model can be used to predict the survival probability of an individual with a specific value of $Z$ at a given point in time $t$ using:

$$S(t|Z=z)=\exp\left(-\int_0^th_0(t)e^{\beta Z}dt\right)$$

To take into account confounding, this can be performed by including a sufficient set of confounders in the Cox model as additional independent variables (with an appropriate functional form) and using this model to perform *regression standardization*, also known as $g$-computation (Keil et al., 2014).

The survival probability at the time $t$ is estimated for each person by using the model and the observed values of the included confounders. The mean of these person specific estimates in a sample of size $n$ is an unbiased estimator of $S_z(t)$.

Formally, this can be defined as:

$$\hat S_z(t)=\frac{1}{n}\sum_{i=1}^{n}S(t|Z=z,X=x_i)$$

This method has been extensively used when analyzing time-to-event data (Makuch, 1982).

Four fundamental assumptiosn have to be met to allow a causal interpretation of the obtained values.
1. First, the no interference assumption states that the potential survival time of an individual is independent of the values of $Z$ of other individuals
2. Second, counterfactual consistency posits that the potential survival time would remain the same under $Z$ regardless of whether $Z$ was set experimentally or not
3. Third, the conditional exchangeability assumption asserts that the treatment groups are exchangeable, given a sufficient set of relevant confounders
4. Positivity - generally not an issue because of the interpolation performed by the time-to-event model.

In addition, any model-specific assumptions, such as the proportional hazards assumption when using a Cox model also have to be met. This includes the correct specification of the functional form of all included variables and the proper treatment of censored observations (e.g., random right-censoring).

## Visualization of the Results

Survival area plots, proposed by this paper, keep the structure of Kaplan-Meier plots with $t$ on the x-axis and the survival probability on the y-axis. Instead of single survival curves for some categories, an area is drawn with the filled color corresponding to values of $Z$.



## Discussion

g-computation is one of many methods to estimate counterfactual survival probabilities. It is a single robust method, which means that it relies on the outcome model to obtain unbiased results. Different authors recommend using doubly-robust methods, such as Augmented-Inverse-probability of Treatment Weighting or Targeted Maximum Likelihood Estimation for similarly defined estimands. However, this is currently not possible, because they do not support continuous exposure.